In [1]:
# Importing the necessary packages
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
import textwrap

In [2]:
# Load multiple PDFs into a list of documents
def load_pdfs(file_paths):
    all_docs = []
    for file_path in file_paths:
        loader = PyMuPDFLoader(file_path=file_path)
        docs = loader.load()
        all_docs.extend(docs)  # Add documents from each PDF to the combined list
    return all_docs

In [3]:
# Responsible for splitting the documents into several chunks
def split_docs(documents, chunk_size=1000, chunk_overlap=20):

    # Initializing the RecursiveCharacterTextSplitter with
    # chunk_size and chunk_overlap
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )

    # Splitting the documents into chunks
    chunks = text_splitter.split_documents(documents=documents)

    # returning the document chunks
    return chunks

In [4]:
# function for loading the embedding model
def load_embedding_model(model_path, normalize_embedding=True):
    return HuggingFaceEmbeddings(
        model_name=model_path,
        model_kwargs={'device':'cpu'}, # here we will run the model with CPU only
        encode_kwargs = {
            'normalize_embeddings': normalize_embedding # keep True to compute cosine similarity
        }
    )

In [5]:
# Function for creating embeddings using FAISS
def create_embeddings(chunks, embedding_model, storing_path="vectorstore"):
    # Creating the embeddings using FAISS
    vectorstore = FAISS.from_documents(chunks, embedding_model)

    # Saving the model in current directory
    vectorstore.save_local(storing_path)

    # returning the vectorstore
    return vectorstore

In [6]:
# Creating the chain for Question Answering
def load_qa_chain(retriever, llm, prompt):
    return RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever, # here we are using the vectorstore as a retriever
        chain_type="stuff",
        return_source_documents=True, # including source documents in output
        chain_type_kwargs={'prompt': prompt} # customizing the prompt
    )

In [7]:
# Prettifying the response
def get_response(query, chain):
    # Getting response from chain
    response = chain.invoke({'query': query})
    
    # Wrapping the text for better output in Jupyter Notebook
    wrapped_text = textwrap.fill(response['result'], width=100)
    print(response['result'])

## Implementation

In [8]:
from langchain_ollama.llms import OllamaLLM
from langchain import PromptTemplate

In [9]:
# llm = OllamaLLM(model="orca-mini", temperature=0)

In [10]:
llm = OllamaLLM(model="llama3.2", temperature=0)

In [11]:
# llm = OllamaLLM(model="mistral", temperature=0)

In [12]:
# embed = load_embedding_model(model_path="all-MiniLM-L6-v2")
embed = load_embedding_model(model_path="sentence-transformers/all-mpnet-base-v2")
# embed = load_embedding_model(model_path="sentence-transformers/multi-qa-mpnet-base-dot-v1")

In [13]:
file_paths = [
    "./docs/rodtep_guidelines-output.pdf",
    "./docs/Drawback-Rates.pdf",
    "./docs/furniture_tv.pdf",
]
docs = load_pdfs(file_paths=file_paths)

In [14]:
documents = split_docs(documents=docs)

# creating vectorstore
vectorstore = create_embeddings(documents, embed)

# converting vectorstore to a retriever
retriever = vectorstore.as_retriever()

In [15]:
# Creating the prompt from the template which we created before
prompt = PromptTemplate.from_template("""
### System:
You are an respectful and honest assistant. You have to answer the user's \
questions using only the context provided to you. If you don't know the answer, \
please think rationally and answer from your own knowledge base

### Context:
{context}

### User:
{question}

### Response:
""")

# Creating the chain
chain = load_qa_chain(retriever, llm, prompt)

In [16]:
get_response("Explain the Kaplan–Meier Survival Curve", chain)

I'd be happy to explain the Kaplan-Meier Survival Curve.

The Kaplan-Meier Survival Curve is a graphical representation of the survival probability of subjects in a study over time, typically used in medical research and statistics. It's named after Edwin Kaplan and Maurice Meier, who first described it in 1958.

The curve shows the proportion of patients who have not experienced an event (such as death, relapse, or recurrence) at different time points. The x-axis represents time, and the y-axis represents the survival probability.

Here's how it works:

1. **Censoring**: In a study, some participants may still be alive at the end of the observation period, but their outcome status (e.g., dead or alive) is not known. These participants are said to be censored.
2. **Event occurrence**: When an event occurs (e.g., death), the participant's survival time is recorded as the time of event occurrence.
3. **Kaplan-Meier estimation**: The Kaplan-Meier estimator calculates the survival probabil

In [17]:
get_response("Explain the RDOTEP Guidelines", chain)

The RoDTEP (Remission of Duties and Taxes on Exported Products) Scheme is a government initiative aimed at refunding duties and taxes levied on exported products. Here's an overview of the guidelines:

**Objective:**
The primary objective of the RoDTEP Scheme is to refund duties and taxes borne by exporters, including prior stage cumulative indirect taxes on goods and services used in production.

**Eligibility:**
The scheme is available for exports made by categories mentioned in para 4.55 (x), (xi), and (xii) above.

**Rebate Nature:**
The rebate under the RoDTEP Scheme will be used only for payment of duty of Customs leviable under the First Schedule to the Customs Tariff Act, 1975, specifically Basic Customs Duty.

**Scheme Structure:**

1. **Output-Outcome Framework:** A broad-level monitoring framework will be maintained and monitored at regular intervals.
2. **Risk Management System (RMS):** An IT-based RMS will be implemented by the Central Board of Indirect Taxes and Customs (

In [18]:
get_response("What are some of the things I have to keep in mind while exporting electronics", chain)

As an assistant, I'll provide you with general guidance on what to keep in mind while exporting electronics, based on the context provided. Please note that specific regulations and requirements may vary depending on the destination country and product specifications.

When exporting electronics, here are some key things to consider:

1. **Compliance with Regulations**: Ensure your products comply with relevant regulations, such as those mentioned in the context (e.g., CARB, EPA, ASTM). Research and understand the specific requirements for your product category.
2. **Labeling and Declaration**: Familiarize yourself with labeling requirements, such as the Federal Hazardous Substances Act (FHSA) label. Declare any hazardous materials or substances used in your products accurately and truthfully.
3. **Export Documentation**: Ensure you have accurate and complete export documentation, including commercial invoices, packing lists, and certificates of origin. Verify that electronic documenta

In [19]:
get_response("What if i want to export a apple iphone mobile to USA", chain)

Based on the context provided, it appears that you are referring to exporting an Apple iPhone mobile from India to the USA.

According to the information available, Apple iPhones manufactured in India would fall under the category of products manufactured partly or wholly in India under Section 65 of the Customs Act, 1962.

As per the Foreign Trade Policy, export of such products is subject to various regulations and requirements. However, I couldn't find specific information on the export procedures for Apple iPhones from India to the USA.

To determine the correct export documentation, sampling procedures, and record-keeping requirements for your Apple iPhone mobile, I would recommend checking with the Central Board of Indirect Taxes and Customs (CBIC) or the Directorate General of Foreign Trade (DGFT), which are responsible for regulating exports in India. They may be able to provide you with more specific guidance on the necessary procedures and documentation required for exporting

In [20]:
get_response("What if i want to export furniture", chain)

If you want to export furniture, there are several requirements and regulations you need to be aware of:

1. **Registration with EPA**: If your furniture contains any antimicrobial textile or other pesticide-related characteristics, it must be registered with the Environmental Protection Agency (EPA).
2. **TSCA Compliance**: As a manufacturer of wooden products, you are required to comply with the formaldehyde emission limits established by the Toxic Substances Control Act (TSCA) Title VI. You need to conduct testing of your product through a CPSCA-affiliated lab and obtain a certificate from the Importer on Record.
3. **Customs Regulations**: Depending on the type of furniture you are exporting, there may be specific categories that apply to it. For example:
	* Wooden frames for paintings, photographs, mirrors, or similar objects (4414) fall under 1.3% duty rate.
	* Packing cases, boxes, crates, drums, and similar packings of wood; cable-drums of wood; pallets, box pallets, and other 

In [21]:
get_response("Give me a checklist if i want to export furniture", chain)

Based on the provided context, here is a checklist for exporting furniture:

**Pre-Export Compliance Requirements:**

1. **Check product classification**: Ensure that your furniture falls under one of the following categories and complies with the relevant regulations:
	* 4414: Wooden frames for paintings, photographs, mirrors or similar objects
	* 4415: Packing cases, boxes, crates, drums and similar packings, of wood; cable-drums of wood; pallets, box pallets and other load boards, of wood; pallet collars of wood
	* 4416: Casks, barrels, vats, tubs and other coopers' products and parts thereof, of wood, including staves
	* 4417: Tools, tool bodies, tool handles, broom or brush bodies and handles, of wood; boot or shoe lasts and trees, of wood
	* 4418: Builders’ joinery and carpentry of wood, including cellular wood panels, assembled flooring panels, Shingles and shakes
	* 4419: Tableware and kitchenware, of wood
2. **Check for antimicrobial textile or pesticide-related characteristic

In [24]:
get_response("What are the rates for furniture?", chain)

Based on the provided tariff item information, I found that there is no specific tariff item for "Furniture" as a whole category. However, some related items have their own tariff rates.

For example:

* Mattress supports (9404) has a rate of 1.3%.
* Pillow/Cushions/Quilts/Pouffles made of cotton fabric filled with poly-fil (940401) has a rate of 2.8% per kg.
* Bed linen, table linen, toilet linen and kitchen linen (6302) has rates ranging from 2.6% to 3.5% per kg for different types of fabrics.

If you're looking for the overall rate for furniture, I couldn't find a specific tariff item that covers this category as a whole. If you have more information or context about the type of furniture you're referring to, I may be able to provide a more accurate answer.


In [23]:
get_response("What are the rates?", chain)

There are two tariff items with rates mentioned:

1. For Tariff Item 41, the rate is 1.3%.
2. For Tariff Item 162, the rate is also 1.3%.

Please note that these rates apply to specific descriptions of goods and may have additional conditions or limitations.
